# INF8775 – Analyse et conception d’algorithmes
# TP1 – Automne 2024

TOURIGNY, François 2079718

CÔTÉ, Gabriel 2082508

Note finale: 

<u>**Date limite de remise :**</u>  1er Octobre 23h59 (Groupe B2), 8 Octobre 23h59 (Groupe B1)

# Instructions

## Rédaction et remise du rapport

- Ce notebook constitue à la fois le sujet du TP, votre code et votre rapport. Il contient déjà du code pour faciliter vos mesures et l'affichage de vos résultats, ainsi qu'un squelette pour votre rapport.

- Complétez directement le notebook, vous êtes libres de créer des nouvelles cellules de code ou de texte. 

- Les questions et tâches à effectuées sont généralement indiquées par un TODO, mais lisez attentivement car nous pourrions avoir oublié d'en indiquer certaines.

- Des questions sont réutilisées d'un algorithme à l'autre (puisque l'on reproduit les expérimentations à des fins de comparaison). Veillez à suffisament développer les premières réponses afin de l'on comprenne bien votre raisonnement et pour montrer votre bonne compréhension. Vous pourrez être plus concis par la suite.

- <u>**IMPORTANT**</u> Remettez le fichier du notebook sur Moodle avec le nom `MATRICULE1_MATRICULE2.ipynb`

- Vous pouvez inclure du code trouvé sur Internet, mais vous devez en mentionner la source, sous peine d'être sanctionnés pour plagiat.

## Mise en situation

Ce travail pratique se répartit sur deux séances de laboratoire et porte sur l’analyse empirique et hybride des algorithmes. Dans les capsules vidéo de la semaine 3, trois approches d’analyse de l’implantation d’un algorithme sont décrites. Vous les mettrez en pratique pour des algorithmes de résolution d’un problème connu.


## Description du problème

On vous demande de résoudre le problème classique de trier une liste de nombres aléatoires.

La taille de la liste ainsi que le nombre maximal permi changeront d'un échantillon à l'autre. Vous pouvez utilisez le code fourni pour générer plus d'instances, mais on fournit aussi des instances intéressantes pour pousser votre réflexion plus loin.

## Algorithmes à implanter

On vous demande de résoudre ce problème de 5 façons différentes :

1. En utilisant un algorithme simple: `Bubble Sort` ;
2. En utilisant une amélioration de l'algorithme précédent: `Cocktail Shaker Sort` ;
3. En utilisant un algorithme diviser pour régner: `Merge Sort` ;
4. En utilisant l'algorithme précédent avec un seuil de récursivité non élémentaire ;
5. En utilisant un algorithme non comparatif: `Radix Sort`.

Pour l’algorithme 4, vous devrez déterminer un seuil de récursivité expérimentalement. Les exemplaires dont la taille est inférieure à ce seuil ne sont plus résolus récursivement mais plutôt directement avec l’algorithme 2.

Pour l'algorithme 5, on vous demande de comparer les chiffres les moins significatifs en premier (`Radix Sort LDS`).


## Jeu de données

La classe Problem existe pour simplifier l'interface des différentes fonctions utilitaires. Elle permet de générer des jeux de données avec la méthode `generate_sample` ci-dessous. Elle génère une liste d'une taille donnée contenant des nombres entre 1 et le maximum permis. Vous pouvez utilisez des listes aléatoires pour tester votre code.

In [139]:
import random
from collections.abc import Iterable

class Problem():
    def __init__(self, size: int, max_number: int, num_samples: int = 5) -> None:
        self.size = size
        self.max_number = max_number
        self.num_samples = num_samples
    
    def generate_sample(self) -> list[int]:
        """Returns a list of the given size containing numbers between 1 and the max_number"""
        return [random.randint(1,self.max_number) for _ in range(self.size)]
    
    def generate_dataset(self) -> Iterable[list[int]]:
        """Returns an iterator over as many samples as are described """
        return (self.generate_sample() for _ in range(self.num_samples))

# Implantations et expérimentations

Ces fonctions auxiliaires vous sont fournies pour vérifier l'exactitude des vos algorithmes, mesurer leurs performance et afficher vos résultats.

Il est recommandé de prendre le temps de lire et comprendre le code.

Exécutez la cellule ci-dessous pour pouvoir utiliser les fonctions auxiliaires.

In [140]:
import matplotlib.pyplot as plt
import time
from collections.abc import Callable
from math import log10
from scipy.stats import linregress

class InvalidSolution(Exception):
    def __init__(self):
        super().__init__("Invalid solution, verify your code.")

class Measure():
    """A wrapper to contain information on taken measures"""
    def __init__(self, size: int, max_number: int, mean: int) -> None:
        self.size = size
        self.max_number = max_number
        self.mean = mean

def convert_list_to_dict(original: list[int]) -> dict[int:int]:
    """Converts a list into a dictionary of frequencies"""
    freq: dict[int:int] = dict()
    for iter in original:
        if iter not in freq.keys():
            freq[iter] = 0
        freq[iter] += 1
    return freq

def is_valid_solution(original: list[int], solution: list[int]) -> bool:
    """Validates both if the solution is sorted and if the list was not modified"""
    # Lists must be of equal length
    if len(solution) != len(original):
        return False
    
    # List must be in increasing order
    for i in range(1, len(solution)):
        if solution[i-1] > solution[i]:
            return False
    
    original_freq = convert_list_to_dict(original)
    solution_freq = convert_list_to_dict(solution)
    # Lists must have the same values
    for key in original_freq.keys():
        if key not in solution_freq.keys() or\
            solution_freq[key] != original_freq[key]:
            return False
    
    # Solution is valid
    return True

def make_problems(sizes: list[int], max_numbers: list[int], num_samples: int = 5) -> list[Problem]:
    """Creates problem instances using given sizes and max_numbers"""
    problems: list[Problem] = []
    for size in sizes:
        for max_number in max_numbers:
            problems.append(Problem(size,max_number,num_samples))
    return problems

def measure(procedure: Callable[[list[int]],list[int]], sample: list[int], time_scale: int = 1000) -> int:
    """Returns the time in milliseconds taken to run the procedure.

    Raises:
        InvalidSolution: If the procedure returns an invalid solution, raises an exception.
    """
    start: int = time.time() * time_scale
    solution: list[int] = procedure(sample)
    end: int = time.time() * time_scale
    if not is_valid_solution(sample, solution):
        raise InvalidSolution()
    return round(end - start)

def measure_mean(procedure: Callable[[list[int]],list[int]], prob: Problem, time_scale: int = 1000) -> Measure:
    """Generates multiple samples with the specified parameters and returns the mean time in milliseconds

    Raises:
        InvalidSolution: If one of the samples results in an invalid solution.
    """
    mean_time = sum(
        [measure(procedure,sample,time_scale) for sample in prob.generate_dataset()]
    ) / prob.num_samples
    return Measure(prob.size, prob.max_number, mean_time)

def measure_range(procedure: Callable[[list[int]],list[int]], problems: list[Problem], time_scale: int = 1000) -> list[Measure]:
    """Measures the mean time taken in milliseconds for each size in the given list.
    Uses a fixed max_number to measure the effect of size on its own.

    Raises:
        InvalidSolution: If one of the samples results in an invalid solution.
    
    Returns:
        A list of Measure instances containing the specifications
        of the problem as well as the mean time.
    """
    return [
        measure_mean(procedure, prob, time_scale)
        for prob in problems
    ]

def test_threshold(
    procedure: Callable[[list[int],int],list[int]],
    thresholds: list[int], problem: Problem, time_scale: int = 1000
) -> dict[int,int]:
    """Tests the different thresholds on the same problem instance."""
    threshold_measures = {t:0 for t in thresholds}
    for sample in problem.generate_dataset():
        for t in thresholds:
            start = time.time() * time_scale
            solution = procedure(sample,t)
            end = time.time() * time_scale
            if not is_valid_solution(sample,solution):
                raise InvalidSolution()
            threshold_measures[t] += (end - start) / problem.num_samples
    return threshold_measures

def estimate_threshold(
    first_data: dict[int,int],
    second_data: dict[int,int],
    first_label: str,
    second_label: str,
    y_label: str
):
    plt.plot(list(first_data.keys()),list(first_data.values()),label=first_label)
    plt.plot(list(second_data.keys()),list(second_data.values()),label=second_label)
    plt.xlabel('Taille')
    plt.ylabel(y_label)
    plt.title('Estimation du seuil')
    plt.show()

def display_threshold_measures(data: dict[int,int]):
    """Displays a graph of the time take to solve in regards to the chosen threshold"""
    x = list(data.keys())
    y = list(data.values())
    plt.plot(x, y, label='Mesures')
    plt.scatter(x, y, label='Mesures')

    # Add labels and title
    plt.xlabel('Seuil')
    plt.ylabel('Temps (ms)')
    plt.title('Selection du seuil')
    plt.show()
     
def display_data_as_table(measures: list[Measure]):
    """Prints a table with the data in the given list of measures"""
    print("{: <12} {: <12} {: <12}".format("Taille", "Nombre maximal", "Temps moyen (ms)"))
    for measure in measures:
        print("{: <12} {: <12} {: <12}".format(measure.size, measure.max_number, measure.mean))

### The different tests are below, the names are in french to avoid confusion

def test_de_puissance(
    data: dict[int,int],
    x_label: str,
    y_label: str,
    title: str = "Test de puissance"
):
    """Takes the data and displays it into the corresponding test graph.
    It applies no transformations to the data.

    Args:
        data (dict[int,int]): A dictionnary mapping the x variable to the y variable
    """
    # Log both sets of values
    x = list(data.keys())
    y = list(data.values())
    
    # Perform the lin regression
    m, b, rvalue, _, _ = linregress(x, y)
    
    # Estimate the values of y based on the lin regression results
    predicted = [m * iter + b for iter in x]

    # Create the line equation
    line_eq = f"y = {m:.2f}x + {b:.2f}"
    
    # Plot the points
    plt.scatter(x, y, label='Mesures')

    # Plot the regression line
    plt.plot(x, predicted, color="red", label=f'Regression linéaire R²={round(rvalue**2,6)}')
    
    # Add labels and title
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)

    # Add legend
    plt.legend(bbox_to_anchor=(0.60, 0), loc='lower left')

    # Display the line equation
    plt.text(min(x), max(y), line_eq)

    # Show the plot
    plt.show()

def test_de_rapport(
    data: dict[int,int],
    x_label: str,
    y_label: str,
    title: str = "Test de rapport"
):
    """Takes the data and displays it into the corresponding test graph.
    It applies no transformations to the data.

    Args:
        data (dict[int,int]): A dictionnary mapping the x variable to the y variable
    """
    x = list(data.keys())
    y = list(data.values())

    plt.plot(x, y, label='Mesures')
    plt.scatter(x, y, label='Mesures')

    # Add labels and title
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.show()

def test_de_constantes(
    data: dict[int,int],
    x_label: str,
    y_label: str = "Temps (ms)",
    title: str = "Test de constantes"
):
    """Takes the data and displays it into the corresponding test graph.
    It applies no transformations to the data.

    Args:
        data (dict[int,int]): A dictionnary mapping the x variable to the y variable
    """
    x = list(data.keys())
    y = list(data.values())

    # Perform linear regression
    m, b, rvalue, _, _ = linregress(x, y)

    predicted = [m * iter + b for iter in x]

    # Create the line equation
    line_eq = f"y = {m:.2E}x + {b:.2E}"

    # Plot the points
    plt.scatter(x, y, label='Mesures')

    # Plot the regression line
    plt.plot(x, predicted, color="red", label=f'Regression linéaire R²={round(rvalue**2,6)}')

    # Add labels and title
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)

    # Add legend
    plt.legend(bbox_to_anchor=(0.60, 0), loc='lower left')

    # Display the line equation
    plt.text(min(x), max(y), line_eq)

    # Show the plot
    plt.show()

## Partie 1 : Algorithme simple (Bubble Sort) (5 pts)

### Implantation

<u>**Question 1.a):**</u> Implantez l'algorithme de tri bubble sort.

Utilisez la fonction `is_valid_solution` pour valider votre réponse sur quelques exemplaires aléatoires.

In [141]:
def bubble_sort(original: list[int]) -> list[int]:
    # TODO Bubble sort
    swap = True
    while swap:
        swap = False
        for i in range(len(original)-1):
            if original[i] > original[i+1]:
                swap = True
                temp = original[i]
                original[i] = original[i+1]
                original[i+1] = temp
    return original

In [ ]:
problem = Problem(10, 100)
sample = problem.generate_sample()
is_valid_solution(sample, bubble_sort(sample))

<u>**Question 1.b):**</u> Quelle est la complexité asymptotique théorique de cet algorithme? Veuillez justifier.

La complexité de cet algorithme est thétha de n^2 parce que pour chaque passage, on passe une fois à travers les n nombres et on va avoir à replacer les n nombres au bon endroit. Le pire cas étant quand la liste contient le nombre le plus petit à la fin alors on doit prendre chacun des nombres et aller les porter après les nombre le plus plus petit donc n fois n passages ce que nous donne une complexité de n^2.

### Mesures

Pour cet algorithme ainsi que les prochains, vous devez choisir les différentes tailles de liste que vous voulez tester. Choisissez des tailles qui ont des résultats intéressants, une taille de 5 qui se termine en une fraction de millisecondes n'est pas le meilleur résultat pour faire des comparaisons.

Pour faire des mesures, utilisez la fonction `measure_range`. Elle permet de faire des mesures sur une liste de problèmes. Vous pouvez facilement créer des problèmes en utilisant la fonction `make_problems`. Ne faites varier que la taille <u>**OU**</u> le nombre maximal pour observer le comportement d'un des deux paramètres. Le code fournit s'occupera de générer des échantillons aléatoires, de calculer le temps moyen et de vous retourner une liste de mesures. Vous pouvez utilisez la fonction `range` de python pour obtenir une grande liste et avoir plusieurs points (faites attention au temps d'exécution).

Ces données peuvent ensuite être passées aux fonctions `test_de_...` en les mettant dans un `dict` python tel que `x:y`. Les tests ont de la documentation pour expliquer leur utilisation en plus de détails.

<u>**Question 1.c):**</u> Faites afficher vos mesures dans un tableau avec la fonction `display_data_as_table`.

In [63]:
# TODO take measurements
results_size_change = measure_range(bubble_sort, make_problems([10*x for x in range(1, 101)], [10000 for _ in range(10)]))

In [64]:
results_max_number = measure_range(bubble_sort, make_problems([500 for _ in range(10)], [100*x for x in range(1, 101)]))

In [65]:
def dict_size_from_measures(measures: list[Measure]) -> dict:
    dict = {}
    for measure in measures:
        dict[measure.size] = measure.mean
    return dict

In [66]:
def dict_max_number_from_measures(measures: list[Measure]) -> dict:
    dict = {}
    for measure in measures:
        dict[measure.max_number] = measure.mean
    return dict

In [67]:
dict_results_size = dict_size_from_measures(results_size_change)
dict_results_max_number = dict_max_number_from_measures(results_max_number)

In [ ]:
# TODO Display data as tables
display_data_as_table(results_size_change)
print(dict_results_size)
display_data_as_table(results_max_number)
print(dict_results_max_number)

### Analyse Hybride

#### Test de puissance

<u>**Question 2.a):**</u> Effectuez le test de puissance de votre algorithme.

In [ ]:
# TODO Test de puissance
test_de_puissance(dict_results_size, "nombre d'éléments dans la liste", 'temps moyen de tri (ms)', "Test de puissance Bubble sort nombre d'éléments")
test_de_puissance(dict_results_max_number, "nombre maximal", 'temps moyen de tri (ms)', 'Test de puissance Bubble sort valeurs max')

<u>**Question 2.b):**</u> Analysez le graphe obtenu pour le test de puissance.

Avec les graphiques obtenus nous voyons que la puissance semble être définie par une relation exponentielle. En effet, l'augmentation du nombre d'éléments croit plus rapidement qu'une relation polynomiale. Au niveau du changement de la grosseur des nombres utilisés, on ne voit aucune augmentation du temps de tri par rapport aux valeurs. Le rapport entre le temps de tri et la grosseur des chiffres serait linéaire comme il n'y a aucune incidence entre la valeur maximale et le temps de tri.

#### Test de rapport

<u>**Question 3.a):**</u> Effectuez le test de rapport de votre algorithme.

In [ ]:
# TODO Test de rapport
def change_to_predict(init_dict: dict, hypothesis = lambda x: x) -> dict:
    new_dict = {}
    for key in init_dict:
        new_dict[key] = init_dict[key]/hypothesis(key)
    return new_dict

test_de_rapport(change_to_predict(dict_results_size, lambda x: x**2), "nombre d'éléments dans la liste", 'temps de tri divisé par hypothèse de relation exponentielle', "Test de rapport Bubble sort nombre d'éléments au carre(n^2)")

# le test de rapport pour la valeur maximale n<est pas pertinent comme nous avons une relation qui ne dépend pas de x. Il est donc impossible de faire un graphe
#test_de_rapport(change_to_predict(dict_results_max_number, lambda x: 1), "nombre maximal", 'temps de tri divisé par hypothèse de relation constante', 'Test de rapport Bubble sort valeurs max correlation de O(1)')

<u>**Question 3.b):**</u> Analysez le graphe obtenu pour le test de rapport.

On voit que les valeurs semblent se rendre vers une valeur et converger. On voit donc que notre hypothèse de relation exponentielle semble bonne comme il existe une constante vers lequel on converge.

#### Test des constantes

<u>**Question 4.a):**</u> Effectuez le test des constantes de votre algorithme.

In [ ]:
# TODO Test des constantes

def change_x_to_predict(init_dict: dict, hypothesis = lambda x: x) -> dict:
    new_dict = {}
    for key in init_dict:
        new_dict[hypothesis(key)] = init_dict[key]
    return new_dict

test_de_constantes(change_x_to_predict(dict_results_size, lambda x: x**2), "nombre d'éléments", 'temps moyen de tri (ms)', "Test de constantes Bubble sort nombre d'éléments")
#
# le test de constante n<est pas pertinent pour une relation lineaire et donc deja constante
# test_de_constantes(change_x_to_predict(dict_results_max_number, lambda x: 1), "grosseur maximale des nombres", 'temps moyen de tri (ms)', 'Test de constantes Bubble sort valeurs max')

<u>**Question 4.b):**</u> Analysez le graphe obtenu pour le test des constantes.

Selon le test de constante que nous avons, la relation de puissance serait très près d'être exactement n au carré. En effet, la constante obtenue est tout près de 1.

## Partie 2 : Algorithme amélioré (Cocktail Sort) (5 pts)

### Implantation

<u>**Question 1.a):**</u> Implantez l'algorithme de tri cocktail sort.

Utilisez la fonction `is_valid_solution` pour valider votre réponse sur quelques exemplaires aléatoires.

In [72]:
def cocktail_sort(original: list[int]) -> list[int]:
    # TODO
    swap = True
    while swap:
        swap = False
        for i in range(len(original)-1):
            if original[i] > original[i+1]:
                swap = True
                temp = original[i]
                original[i] = original[i+1]
                original[i+1] = temp
        if not swap:
            break
        for i in range(len(original)-1, 1, -1):
            if original[i] < original[i-1]:
                swap = True
                temp = original[i]
                original[i] = original[i-1]
                original[i-1] = temp
    return original

In [ ]:
problem = Problem(10, 100)
sample = problem.generate_sample()
is_valid_solution(sample, cocktail_sort(sample))

<u>**Question 1.b):**</u> Quelle est la complexité asymptotique théorique de cet algorithme? Veuillez justifier.

### Mesures

<u>**Question 1.c):**</u> Faites afficher vos mesures dans un tableau avec la fonction `display_data_as_table`.

In [74]:
# TODO take measurements
results_size_change_cocktail = measure_range(cocktail_sort, make_problems([10*x for x in range(1, 101)], [10000 for _ in range(10)]))

In [75]:
results_max_number_cocktail = measure_range(cocktail_sort, make_problems([500 for _ in range(10)], [100*x for x in range(1, 101)]))

In [76]:
dict_results_size_cocktail = dict_size_from_measures(results_size_change_cocktail)
dict_results_max_number_cocktail = dict_max_number_from_measures(results_max_number_cocktail)

In [ ]:
# TODO Display data as tables
display_data_as_table(results_size_change_cocktail)
print(dict_results_size_cocktail)
display_data_as_table(results_max_number_cocktail)
print(dict_results_max_number_cocktail)

### Analyse Hybride

#### Test de puissance

<u>**Question 2.a):**</u> Effectuez le test de puissance de votre algorithme.

In [ ]:
# TODO Test de puissance
test_de_puissance(dict_results_size_cocktail, "nombre d'éléments dans la liste", 'temps moyen de tri (ms)', "Test de puissance Cocktail sort nombre d'éléments")
test_de_puissance(dict_results_max_number_cocktail, "nombre maximal", 'temps moyen de tri (ms)', 'Test de puissance Cocktail sort valeurs max')

<u>**Question 2.b):**</u> Analysez le graphe obtenu pour le test de puissance.

Comme pour le tri par bulle, nous voyons que la relation linéaire n'est pas la bonne. On remarque que le temps de tri augmente beaucoup plus rapidement que le nombre d'exemplaire à trier. Le tri cocktail serait donc aussi une relation exponentielle entre le nombre d'éléments et le temps de tri. Au niveau du nombre de valeurs ça ne semble pas influencer du tout le temps de tri. Bref, le temps de tri serait constant par rapport à la grosseur des nombres.

#### Test de rapport

<u>**Question 3.a):**</u> Effectuez le test de rapport de votre algorithme.

In [ ]:
# TODO Test de rapport
test_de_rapport(change_to_predict(dict_results_size_cocktail, lambda x: x**2), "nombre d'éléments dans la liste", 'temps de tri divisé par le rapport exponentiel de n^2', "Test de rapport Cocktail sort nombre d'éléments")
#test_de_rapport(dict_results_max_number_cocktail, "nombre maximal", 'temps moyen de tri (ms)', 'Test de rapport Cocktail sort valeurs max')

<u>**Question 3.b):**</u> Analysez le graphe obtenu pour le test de rapport.

On voit comme notre intuition le pr/disait que le rapport semble être exponentiel. En effet, il semble y avoir une convergence vers une valeur constante.

#### Test des constantes

<u>**Question 4.a):**</u> Effectuez le test des constantes de votre algorithme.

In [ ]:
# TODO Test des constantes
test_de_constantes(change_x_to_predict(dict_results_size_cocktail, lambda x: x**2), "nombre d'éléments", 'temps moyen de tri (ms)', "Test de constantes Cocktail sort nombre d'éléments")
#test_de_constantes(dict_results_max_number_cocktail, "grosseur maximale des nombres", 'temps moyen de tri (ms)', 'Test de constantes Cocktail sort valeurs max')

<u>**Question 4.b):**</u> Analysez le graphe obtenu pour le test des constantes.

Tout comme pour le tri par bulle, une valeur de convergence semble être atteinte. Par contre, cette fois-ci la valeur est significativement plus petite que 1. Le tri cocktail est donc de relation de complexité n^2 tout comme le tri par bulle, mais malgré tout beaucoup plus efficace.

## Partie 3 : Algorithme diviser pour régner (Merge Sort) (5 pts)

### Implantation

<u>**Question 1.a):**</u> Implantez l'algorithme de tri merge sort. Cette algorithme utilise le patron de conception "Diviser pour régner".

Utilisez la fonction `is_valid_solution` pour valider votre réponse sur quelques exemplaires aléatoires.

In [81]:
def merge_sort(original: list[int]) -> list[int]:
    # TODO
    if len(original) <= 1:
        return original
    #split
    left = merge_sort(original[:int(len(original)/2)])
    right = merge_sort(original[int(len(original)/2):])
    #merge
    merge_result = []
    i = 0
    j = 0
    while i < len(left) or j < len(right):
        if i < len(left) and (not j < len(right) or left[i] < right[j]):
            merge_result.append(left[i])
            i += 1
        else:
            merge_result.append(right[j])
            j += 1
    return merge_result

In [ ]:
problem = Problem(10, 100)
sample = problem.generate_sample()
is_valid_solution(sample, merge_sort(sample))

<u>**Question 1.b):**</u> Quelle est la complexité asymptotique théorique de cet algorithme? Veuillez justifier.

### Mesures

<u>**Question 1.c):**</u> Faites afficher vos mesures dans un tableau avec la fonction `display_data_as_table`.

In [90]:
# TODO take measurements
results_size_change_merge = measure_range(merge_sort, make_problems([50*x for x in range(1, 101)], [10000 for _ in range(10)]))

In [84]:
results_max_number_merge = measure_range(merge_sort, make_problems([500 for _ in range(10)], [100*x for x in range(1, 101)]))

In [91]:
dict_results_size_merge = dict_size_from_measures(results_size_change_merge)
dict_results_max_number_merge = dict_max_number_from_measures(results_max_number_merge)

In [ ]:
# TODO Display data as tables
display_data_as_table(results_size_change_merge)
print(dict_results_size_merge)
display_data_as_table(results_max_number_merge)
print(dict_results_max_number_merge)

### Analyse Hybride

#### Test de puissance

<u>**Question 2.a):**</u> Effectuez le test de puissance de votre algorithme.

In [ ]:
# TODO Test de puissance
import math
test_de_puissance(dict_results_size_merge, "nombre d'éléments dans la liste", 'temps moyen de tri (ms)', "Test de puissance Merge sort nombre d'éléments")
test_de_puissance(dict_results_max_number_merge, "nombre maximal", 'temps moyen de tri (ms)', 'Test de puissance Merge sort valeurs max')

<u>**Question 2.b):**</u> Analysez le graphe obtenu pour le test de puissance.

Nous voyons que la relation n'est pas vraiment linéaire entre le nombre d'éléments à classer et le temps pour les trier. En effet, nous supposons que la relation serait plutôt d'ordre n log(n). Pour ce qui est 

#### Test de rapport

<u>**Question 3.a):**</u> Effectuez le test de rapport de votre algorithme.

In [ ]:
# TODO Test de rapport
test_de_rapport(change_to_predict(dict_results_size_merge, lambda x: x * math.log(x)), "nombre d'éléments dans la liste", 'temps moyen de tri (ms)', "Test de rapport Merge sort nombre d'éléments")
#test_de_rapport(dict_results_max_number_merge, "nombre maximal", 'temps moyen de tri (ms)', 'Test de rapport Merge sort valeurs max')

<u>**Question 3.b):**</u> Analysez le graphe obtenu pour le test de rapport.

On voit que malgré les sauts de valeurs, sur une plus grande tendance, nous aurions une convergence vers une valeur. En effet, le ratio augmente, mais toujours de plus en plus lentement, ce qui indique que n log(n) est le bon ordre de croissance du temps de tri selon le nombre d'éléments.

#### Test des constantes

<u>**Question 4.a):**</u> Effectuez le test des constantes de votre algorithme.

In [ ]:
# TODO Test des constantes
test_de_constantes(dict_results_size_merge, "nombre d'éléments", 'temps moyen de tri (ms)', "Test de constantes Merge sort nombre d'éléments")
test_de_constantes(dict_results_max_number_merge, "grosseur maximale des nombres", 'temps moyen de tri (ms)', 'Test de constantes Merge sort valeurs max')

<u>**Question 4.b):**</u> Analysez le graphe obtenu pour le test des constantes.

<u>**Question 4.c):**</u> Quel serait un meilleur choix de seuil? Utilisez la fonction `estimate_threshold` pour avoir une idée où commencer. Analysez le graph résultant et choisissez un seuil de départ.

On recommande de compiler des nouvelles données sur des tailles plus petites. Utilisez un `time_scale` plus grand pour avoir plus de détails (indiquez les unités dans le `y_label`). Une fois que vous avez une idée d'où se situe le point de croisement, vous pouvez refaire le test sur une sélection plus restreinte de tailles en augmentant `num_samples` pour avoir des résultats plus constants.

In [ ]:
# TODO Find a better starting point for the threshold


## Partie 4 : Algorithme diviser pour régner avec seuil arbitraire (Merge Sort) (5 pts)

### Implantation

<u>**Question 1.a):**</u> Reprenez l'algorithme précédent et modifiez-le pour y ajouter un seuil de récursivité. En dessous de ce seuil, vous utiliserez l'algorithme `cocktail_sort` écrit précédemment.

Utilisez la fonction `is_valid_solution` pour valider votre réponse sur quelques exemplaires aléatoires.

In [30]:
def merge_sort_threshold(original: list[int], threshold: int = 1) -> list[int]:
    # TODO
    if len(original) <= threshold:
        return cocktail_sort(original)
    #split
    left = merge_sort(original[:int(len(original)/2)])
    right = merge_sort(original[int(len(original)/2):])
    #merge
    merge_result = []
    i = 0
    j = 0
    while i < len(left) or j < len(right):
        if i < len(left) and (not j < len(right) or left[i] < right[j]):
            merge_result.append(left[i])
            i += 1
        else:
            merge_result.append(right[j])
            j += 1
    return merge_result

<u>**Question 1.b):**</u> À l'aide de ce que vous avez vu dans la section précédente, effectuez les mesures avec plusieurs seuils de récursivité pour déterminer le seuil le plus judicieux. Affichez les dans un graphique une fois que les mesures sont compilées. La fonction `test_threshold` vous sera utile pour faire ces tests. Choisissez bien vos seuils ainsi qu'une taille de liste permettant de bien observer la différence de performance.

In [31]:
# TODO Test different threshold values premiere exploration
problem = Problem(1000, 10000)
threshold_results = test_threshold(merge_sort_threshold, [x for x in range(1,2000, 5)], problem)

In [ ]:
# TODO Display a graph with the measurements
display_threshold_measures(threshold_results)

In [33]:
# Test thresholds more precise exploration
problem = Problem(1000, 10000, 20)
threshold_results = test_threshold(merge_sort_threshold, [x for x in range(980,1020)], problem)

In [ ]:
# Display a graph with the measurements
display_threshold_measures(threshold_results)

<u>**Question 1.c):**</u> Quel est le seuil de récursivité le plus judicieux? Sur quel critère l'avez vous sélectionné? Pourquoi des seuils inférieurs ou supérieurs donnent-ils de moins bonnes performances?

Changez le seuil par défaut de votre fonction au meilleur choix. Ceci vous permettra d'utiliser les fonctions utilitaires sans passer le seuil comme argument.

<u>**Question 1.d):**</u> La complexité asymptotique théorique de cet algorithme a-t-elle changée? Veuillez justifier.

### Mesures

<u>**Question 1.e):**</u> Faites afficher vos mesures dans un tableau avec la fonction `display_data_as_table`.

In [ ]:
# TODO take measurements
results_size_change_merge_threshold = measure_range(merge_sort_threshold, make_problems([100*x for x in range(1, 11)], [10000 for _ in range(10)]))
results_max_number_merge_threshold = measure_range(merge_sort_threshold, make_problems([1000 for _ in range(10)], [1000*x for x in range(1, 11)]))
dict_results_size_merge_threshold = dict_size_from_measures(results_size_change_merge_threshold)
dict_results_max_number_merge_threshold = dict_max_number_from_measures(results_max_number_merge_threshold)
display_data_as_table(results_size_change_merge_threshold)
print(dict_results_size_merge_threshold)
display_data_as_table(results_max_number_merge_threshold)
print(dict_results_max_number_merge_threshold)

### Analyse Hybride

#### Test de puissance

<u>**Question 2.a):**</u> Effectuez le test de puissance de votre algorithme.

In [ ]:
# TODO Test de puissance
test_de_puissance(dict_results_size_merge_threshold, "nombre d'éléments dans la liste", 'temps moyen de tri (ms)', "Test de puissance Merge sort avec threshold nombre d'éléments")
test_de_puissance(dict_results_max_number_merge_threshold, "nombre maximal", 'temps moyen de tri (ms)', 'Test de puissance Merge avec threshold sort valeurs max')

<u>**Question 2.b):**</u> Analysez le graphe obtenu pour le test de puissance.

#### Test de rapport

<u>**Question 3.a):**</u> Effectuez le test de rapport de votre algorithme.

In [ ]:
# TODO Test de rapport
test_de_rapport(dict_results_size_merge_threshold, "nombre d'éléments dans la liste", 'temps moyen de tri (ms)', "Test de rapport Merge sort avec threshold nombre d'éléments")
test_de_rapport(dict_results_max_number_merge_threshold, "nombre maximal", 'temps moyen de tri (ms)', 'Test de rapport Merge sort avec threshold valeurs max')

<u>**Question 3.b):**</u> Analysez le graphe obtenu pour le test de rapport.

#### Test des constantes

<u>**Question 4.a):**</u> Effectuez le test des constantes de votre algorithme.

In [ ]:
# TODO Test des constantes
test_de_constantes(dict_results_size_merge_threshold, "nombre d'éléments", 'temps moyen de tri (ms)', "Test de constantes Merge sort avec threshold nombre d'éléments")
test_de_constantes(dict_results_max_number_merge_threshold, "grosseur maximale des nombres", 'temps moyen de tri (ms)', 'Test de constantes Merge sort avec threshold valeurs max')

<u>**Question 4.b):**</u> Analysez le graphe obtenu pour le test des constantes.

<u>**Question 5):**</u> Commentez l'impact du seuil de récursivité. Que représente t'il pour l'algorithme diviser pour régner et l'algorithme naïf ?

## Partie 5 : Algorithme non comparatif (Radix Sort LSD) (5 pts)

### Implantation

<u>**Question 1.a):**</u> Implantez l'algorithme de tri radix sort. Cette algorithme ne fait pas de comparaison entre les différents éléments de la liste.

Utilisez la fonction `is_valid_solution` pour valider votre réponse sur quelques exemplaires aléatoires.

In [75]:
def radix_sort(original: list[int]) -> list[int]:
    max_num = max(original)
    exp = 1
    while max_num // exp > 0:
        count = [0] * 10
        output = [0] * len(original)

        for num in original:
            index = (num // exp) % 10
            count[index] += 1

        for i in range(1, 10):
            count[i] += count[i - 1]

        for i in range(len(original) - 1, -1, -1):
            index = (original[i] // exp) % 10
            output[count[index] - 1] = original[i]
            count[index] -= 1

        original = output
        exp *= 10

    return original

In [ ]:
problem = Problem(10, 100)
sample = problem.generate_sample()
is_valid_solution(sample, radix_sort(sample))

<u>**Question 1.b):**</u> Quelle est la complexité asymptotique théorique de cet algorithme? Veuillez justifier.

La complexité asymptotique est Theta de (nd), où n est la taille de l'exemplaire et d est le nombre de chiffres dans le plus gros nombre de l'exemplaire. Par exemple, si le plus gros nombre est 57, alors d = 2. On passe d fois dans le premier loop et (n + 10 + n) fois dans les 2e loop. 

<u>**Question 1.c):**</u> Comment cet algorithme ce distingue-t-il des algorithmes précédents? Est-ce toujours un avantage? Expliquez.


Il se distingue car le temps d'exécution dépends aussi du nombre de chiffres que contient le plus grand entier dans la liste, en plus de la longueur de la liste.

C'est un avantage dépendamment de la valeur de d. si d est plus petit que n, alors radix sort devrait battre les algorithmes qui ont une complexité theta(n^2). Si d est plus petit que log n, alors radix bat les algorithmes d'une complexité de theta(n log n). Si d est plus grand que n, alors tous les algorithmes plus hauts devraient en théorie être plus rapides que radix.

### Mesures

<u>**Question 1.d):**</u> Faites afficher vos mesures dans un tableau avec la fonction `display_data_as_table`.

In [ ]:
results_size_change_radix = measure_range(radix_sort, make_problems([100*x for x in range(1, 101)], [10000 for _ in range(10)]))
results_max_number_radix = measure_range(radix_sort, make_problems([1000 for _ in range(10)], [10**x for x in range(1, 51)]))

dict_results_size_radix = dict_size_from_measures(results_size_change_radix)
dict_results_max_number_radix = dict_max_number_from_measures(results_max_number_radix)

display_data_as_table(results_size_change_radix)
print(dict_results_size_radix)
display_data_as_table(results_max_number_radix)
print(dict_results_max_number_radix)

### Analyse Hybride

#### Test de puissance

<u>**Question 2.a):**</u> Effectuez le test de puissance de votre algorithme.

In [ ]:
# TODO Test de puissance
def dict_max_number_to_linear_dict(dict_max_number):
    new_dict = {}
    for key in dict_max_number.keys():
        # print(str(key))
        # print(len(str(key)))
        new_dict[len(str(key))] = dict_max_number[key]
    return new_dict

dict_number_of_digits = dict_max_number_to_linear_dict(dict_results_max_number_radix)
print(dict_number_of_digits)

test_de_puissance(dict_results_size_radix, "nombre d'éléments dans la liste", 'temps moyen de tri (ms)', "Test de puissance radix sort nombre d'éléments")
test_de_puissance(dict_number_of_digits, "nombre maximal", 'temps moyen de tri (ms)', 'Test de puissance radix sort valeurs max')

<u>**Question 2.b):**</u> Analysez le graphe obtenu pour le test de puissance.

Le test de puissance sur l'algorithme radix semble indiquer une relation linéaire sur la taille de l'exemplaire et sur le nombre de chiffres du plus grand nombre.

#### Test de rapport

<u>**Question 3.a):**</u> Effectuez le test de rapport de votre algorithme.

In [ ]:
# TODO Test de rapport
test_de_rapport(change_to_predict(dict_results_size_radix, lambda x: x), "nombre d'éléments dans la liste", 'temps moyen de tri (ms)', "Test de rapport radix sort nombre d'éléments")
test_de_rapport(change_to_predict(dict_number_of_digits, lambda x: x), "nombre maximal", 'temps moyen de tri (ms)', 'Test de rapport radix sort valeurs max')

<u>**Question 3.b):**</u> Analysez le graphe obtenu pour le test de rapport.

Nous avons appliquer la transformation y / f(x), où f(x) = x à nos données. Pour les tailles et pour le nombre de chiffres, nous semblons avoir une convergence vers une valeur > 0, alors on peut dire que notre hypothèse sur f(x) semble être correcte

#### Test des constantes

<u>**Question 4.a):**</u> Effectuez le test des constantes de votre algorithme.

In [ ]:
# TODO Test des constantes
test_de_constantes(change_x_to_predict(dict_results_size_radix, lambda x: x), "nombre d'éléments", 'temps moyen de tri (ms)', "Test de constantes radix sort nombre d'éléments")
test_de_constantes(change_x_to_predict(dict_number_of_digits, lambda x: x), "grosseur maximale des nombres", 'temps moyen de tri (ms)', 'Test de constantes radix sort valeurs max')

<u>**Question 4.b):**</u> Analysez le graphe obtenu pour le test des constantes.

Ici, le graphe sur le test de constances semble indiquer qu'on a en effet une bonne hypothèse de f(x) = x pour chacune des variables, car nous avons une relation linéaire pour les 2 graphes.

# Conclusion (3 pts)

Résumez *brièvement* vos découvertes et analyses, et indiquez dans quelles circonstances vous utiliserez chacun de ces algorithmes.

 ## Autres critères (2 pts)
 Qualité du code / 1 pt

Présentation générale / 1 pt

- Concision
- Qualité du français

Pénalité retard
- -2 pt / journée de retard, arrondi vers le haut. Les TPs ne sont plus acceptés après 3 jours.